In [ ]:
# import pandas as pd

# CSV_PATH = r"..\\data\\ground_truth\\ground_truth.csv"
# CLASSES_PATH = r"..\\data\\classes.txt"

# df = pd.read_csv(CSV_PATH)
# df.columns = [c.strip() for c in df.columns]

# # Split labels and flatten
# all_labels = df['Ground Truth'].dropna().apply(lambda x: [l.strip() for l in x.split(',')])
# unique_labels = set([label for sublist in all_labels for label in sublist])

# # Save to classes.txt
# with open(CLASSES_PATH, "w") as f:
#     for label in sorted(unique_labels):
#         f.write(f"{label}\n")

# print(f"Generated classes.txt with {len(unique_labels)} classes.")


Generated classes.txt with 319 classes.


In [1]:
import os
import cv2
import json
import pandas as pd
import numpy as np
import shutil

# -----------------------------
# Configuration
# -----------------------------
IMAGE_DIR = r"..\\data\\FoodID_Dataset"
PROCESSED_DIR = r"..\\data\\Processed_Images"
ANNOTATION_DIR = r"..\\data\\YOLO_Annotations"
CLASSES_PATH = r"..\\data\\classes.txt"
CSV_PATH = r"..\\data\\ground_truth\\ground_truth.csv"

os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(ANNOTATION_DIR, exist_ok=True)

HEADER_HEIGHT = 100  # space above image for instructions

# -----------------------------
# Load ground truth CSV
# -----------------------------
df = pd.read_csv(CSV_PATH)
df.columns = [c.strip() for c in df.columns]

# -----------------------------
# Load or generate classes.txt
# -----------------------------
if os.path.exists(CLASSES_PATH):
    with open(CLASSES_PATH, "r") as f:
        class_list = [line.strip() for line in f.readlines()]
else:
    # Generate classes.txt from CSV
    all_labels = df['Ground Truth'].dropna().apply(lambda x: [l.strip() for l in x.split(',')])
    unique_labels = sorted(set([label for sublist in all_labels for label in sublist]))
    with open(CLASSES_PATH, "w") as f:
        for label in unique_labels:
            f.write(f"{label}\n")
    class_list = unique_labels
    print(f"Generated classes.txt with {len(unique_labels)} classes.")

# -----------------------------
# Helper: split labels
# -----------------------------
def safe_split_labels(label_str):
    if isinstance(label_str, str) and label_str.strip():
        return [l.strip() for l in label_str.split(',')]
    else:
        return []

df['Ground Truth'] = df['Ground Truth'].apply(safe_split_labels)

# -----------------------------
# Global variables
# -----------------------------
points = []
polygons = []
current_label = None
clone = None
mask = None

# -----------------------------
# Mouse callback
# -----------------------------
def click_event(event, x, y, flags, param):
    global points, polygons, current_label, clone, mask

    # Ignore clicks in the header
    if y < HEADER_HEIGHT:
        return

    y_corrected = y - HEADER_HEIGHT

    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y_corrected))
    elif event == cv2.EVENT_RBUTTONDOWN:
        if len(points) >= 3:
            polygons.append({
                "label": current_label,
                "points": points.copy()
            })
            cv2.fillPoly(mask, [np.array(points, dtype=np.int32)], 255)
            cv2.polylines(clone, [np.array(points, dtype=np.int32)], isClosed=True, color=(0,0,255), thickness=2)
            points.clear()

# -----------------------------
# Overlay instructions + live polygon
# -----------------------------
def draw_overlay_with_header(image, label_name):
    # Create new canvas with header
    overlay = np.zeros((image.shape[0]+HEADER_HEIGHT, image.shape[1], 3), dtype=np.uint8)
    overlay[:HEADER_HEIGHT, :, :] = (50,50,50)  # header background
    overlay[HEADER_HEIGHT:, :, :] = image  # original image

    # Draw instructions
    instructions = [
        f"Draw polygons for: {label_name}",
        "Left click = add points",
        "Right click = finish polygon",
        "ESC = finish this label"
    ]
    y0, dy = 25, 25
    for i, line in enumerate(instructions):
        y = y0 + i*dy
        cv2.putText(overlay, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)

    # Draw current polygon live
    if len(points) > 0:
        pts = np.array([(x, y+HEADER_HEIGHT) for (x,y) in points], np.int32)
        if len(points) > 1:
            cv2.polylines(overlay, [pts], isClosed=False, color=(0,255,0), thickness=2)
        for p in pts:
            cv2.circle(overlay, p, 3, (0,255,0), -1)

    return overlay

# -----------------------------
# Main annotation loop
# -----------------------------
for idx, row in df.iterrows():
    image_name = str(row['Image #']).strip()
    image_path = os.path.join(IMAGE_DIR, image_name)

    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        continue

    image = cv2.imread(image_path)
    if image is None:
        print(f"Cannot load image: {image_path}")
        continue

    clone = image.copy()
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    polygons = []

    cv2.namedWindow("Draw polygons")
    cv2.setMouseCallback("Draw polygons", click_event)

    labels = row['Ground Truth']

    for lbl in labels:
        current_label = lbl
        if lbl not in class_list:
            print(f"Warning: label '{lbl}' not in classes.txt. Skipping.")
            continue

        points.clear()
        while True:
            display_img = draw_overlay_with_header(clone, current_label)
            cv2.imshow("Draw polygons", display_img)
            key = cv2.waitKey(1) & 0xFF
            if key == 27:  # ESC pressed → next label
                points.clear()
                break

    cv2.destroyAllWindows()

    if not polygons:
        print(f"No polygons drawn for image: {image_name}, skipping YOLO txt creation.")
        continue

    # -----------------------------
    # Save YOLO annotation
    # -----------------------------
    txt_filename = os.path.splitext(image_name)[0] + ".txt"
    txt_path = os.path.join(ANNOTATION_DIR, txt_filename)

    img_h, img_w = image.shape[:2]
    with open(txt_path, "w") as f:
        for poly in polygons:
            label_idx = class_list.index(poly["label"])
            pts = np.array(poly["points"])
            x_min = pts[:,0].min()
            x_max = pts[:,0].max()
            y_min = pts[:,1].min()
            y_max = pts[:,1].max()
            x_center = (x_min + x_max) / 2 / img_w
            y_center = (y_min + y_max) / 2 / img_h
            w = (x_max - x_min) / img_w
            h = (y_max - y_min) / img_h
            f.write(f"{label_idx} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

    # -----------------------------
    # Save polygons JSON
    # -----------------------------
    json_filename = os.path.splitext(image_name)[0] + "_polygons.json"
    json_path = os.path.join(ANNOTATION_DIR, json_filename)
    with open(json_path, "w") as f_json:
        json.dump(polygons, f_json)

    # -----------------------------
    # Move processed image
    # -----------------------------
    shutil.move(image_path, os.path.join(PROCESSED_DIR, image_name))
    print(f"Processed image: {image_name}, polygons saved, moved to processed folder.\n")


Processed image: 1725353808877_image_data.jpg, polygons saved, moved to processed folder.



KeyboardInterrupt: 